<a href="https://colab.research.google.com/github/Tubhalooter/codeschool/blob/tss/TSS_JulyProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the 30 days of code challenge!
___
Feel free to add your code files you have so far in the cells below.
If you need to install any packages for python, you can do below in a code cell by running:
```
!pip install package-name --upgrade --quiet
```
--upgrade is added so it installs the latest package versions and --quiet so it doesn't spam all over the output cell, it will only show error messages

In [27]:
!pip install selenium

In [28]:

#ls code
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime

# libraries imported^

# what page to use
url = 'https://spaceflightnow.com/launch-schedule/'

# request html text from page
html_text = requests.get(url).text

# lxml as parser(finds and then structures/makes sense of html text) to extract html code
html_S_code = BeautifulSoup(html_text, 'lxml')

# this uses the line above and we put in these arguments to then find the line of html code from the page

missionDate = html_S_code.find_all('div', class_="datename")
# to find mission date   # ^ find_all will find all instances of that type inside the brackets


missionData = html_S_code.find_all('div', class_="missiondata")
# to find info of mission

missionDesc = html_S_code.find_all('div', class_="missdescrip")
# to find description of mission

# the three variables above now hold all the lines of html code that match with their arguments


map = {}
id = -1  # for dictionary index counter inside for loop
num_mission = len(missionDate)

for i in range(num_mission):
    dateC = missionDate[i]
    dataC = missionData[i]
    descC = missionDesc[i]

    id += 1
    map[id] = {}

    lDate = dateC.text
    lData = dataC.text
    lDesc = descC.text

    # mapping to a location(id) in the dictionary below
    map[id]['Mission Date'] = lDate
    map[id]['Mission Data'] = lData
    map[id]['Mission Description'] = lDesc
    map[id]['mission num'] = []  # here we have created an empty list that is inside the dictionary under the category mission num, list in a dictionary
    # all of these^ are in the same location in the dictionary, just under different categories in same location

    # all of our scraped values are in a dictionary right now and need to be converted to a df
    # we need to convert to a list of lists to do this

    list = []  # list created

for id in range(len(missionDate)):  # the reason this works is because the parameter inside the len() points to a list,
    # len() of a list gives amount of values
    list.append([])  # appending the list

    list[id].append(str(map[id][
                            'Mission Date']))  # appending a certain value of the list with the string version of the value in (map[id]['missionData'])
    list[id].append(str(map[id]['Mission Data']))
    list[id].append(str(map[id]['Mission Description']))

events_to_df = pd.DataFrame(list, columns=['Mission Date', 'Mission Info', 'Mission Desc'])

MissionFile = 'MissionFiles.csv'
events_to_df.to_csv(MissionFile)

events_df = pd.read_csv('MissionFiles.csv')
events_df

cleaned_data = events_df.replace('\n', ' ', regex=True)# to replace data in a data frame, pretty self explanatory



In [29]:
import urllib.parse
from bs4 import BeautifulSoup


launchSiteList=[]
for listloc in range(len(missionData)):
    launchSiteList.append([])
    find_launch_sites = str(events_df['Mission Info'][listloc]).split('\n')[1]
    launchSiteList[listloc].append(find_launch_sites)


LaunchSite_dataframe = pd.DataFrame(launchSiteList, columns = ['Launch Sites'])
LaunchS_File = 'Launch Sites List.csv'
LaunchSite_dataframe.to_csv(LaunchS_File)

LaunchS_File_R = pd.read_csv('Launch Sites List.csv')

In [30]:
import geopy
from geopy import ArcGIS
from geopy.extra.rate_limiter import RateLimiter
LaunchS_File_R

LSlistDF=pd.read_csv('Launch Sites List.csv')

locator = ArcGIS()
geocode = RateLimiter(locator.geocode,min_delay_seconds=1)
LSlistDF['Co-ordF'] = LSlistDF['Launch Sites'].apply(geocode)
LSlistDF['point'] = LSlistDF['Co-ordF'].apply(lambda loc: tuple(loc.point) if loc else None)

co_ordDF=pd.DataFrame(LSlistDF['point'].tolist(), index=LSlistDF.index,columns=["latitude","longitude",'altitude'])


In [31]:
alldata = pd.concat([cleaned_data,co_ordDF], axis=1)


In [32]:
import folium
LSmaps = folium.Map(
    location=[35.12757451779675, -13.04571377201154],
    tiles='cartoDB dark_matter',
    zoom_start=3,
    min_zoom = 3,
    )

alldata.apply(lambda row:folium.vector_layers.CircleMarker(location=tuple([row['latitude'],row['longitude']]),tooltip=([row['Mission Date'],row['Mission Info'],row['Mission Desc']])).add_to(LSmaps),axis=1)
LSmaps